In [ ]:
import os
import glob
import shutil
import pandas as pd

# Function to export data
def export_data(df, filename, format):
    if format == "csv":
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename} in CSV format.")
    elif format == "json":
        df.to_json(filename, orient="records")
        print(f"Data exported to {filename} in JSON format.")
    else:
        print("Unsupported format")

# Function to move exported files to backup folder
def move_exported_files():
    backup_folder = "backup_folder"
    os.makedirs(backup_folder, exist_ok=True)  
    
    # Move CSV and JSON files
    for file in glob.glob("output.*"):  
        shutil.move(file, backup_folder)
        print(f"Moved file: {file} to {backup_folder}/")

# Sample data
data = {
    'First Name': ['Andrew', 'Lauren', 'Sophia'],
    'Last Name': ['Goodman', 'Villa', 'Mathis'],
    'Country': ['Macao', 'Fiji', 'Switzerland'],
    'Company': ['Stewart-Flynn', 'French, Travis and Hensley', 'Richard-Velasquez']
}

df = pd.DataFrame(data)

# Export data
export_data(df, "output.csv", "csv")
export_data(df, "output.json", "json")

# Move exported files
move_exported_files()

Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.
Moved file: output.csv to backup_folder/
Moved file: output.json to backup_folder/


In [4]:
import sqlite3
import time
import yfinance as yf
import pandas as pd

# Database setup
db_name = "stocks.db"

# Create database connection
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Create table if it does not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER
    )
''')
conn.commit()

# Function to fetch stock data
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None  # Return None if no data is available

        latest = data.iloc[-1]  # Get the most recent price data
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to store data in SQLite
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)
    
    if stock_data:
        cursor.execute('''
            INSERT INTO stock_data (symbol, open, high, low, close, volume) 
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (stock_data["symbol"], stock_data["open"], stock_data["high"], 
              stock_data["low"], stock_data["close"], stock_data["volume"]))
        
        conn.commit()
        print(f"Stored data for {symbol}")

# Function to analyze stock data
def analyze_stock(symbol):
    df = pd.read_sql_query(
        "SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100",
        conn, params=(symbol,)
    )
    print(df)

# Example Usage
symbol = "JPM"  # JP Morgan Chase

for _ in range(5):  # Fetch data 5 times with 1-minute intervals
    store_data(symbol)
    time.sleep(60)  # Wait for 1 minute before fetching again

# Analyze stock data
analyze_stock(symbol)

# Close database connection
conn.close()


Failed to get ticker 'JPM' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002AF10BDB620>: Failed to resolve 'fc.yahoo.com' ([Errno 11002] getaddrinfo failed)"))


Stored data for JPM
Stored data for JPM
Stored data for JPM
Stored data for JPM
Stored data for JPM
   id symbol            timestamp        open        high         low  \
0   5    JPM  2025-03-04 14:12:58  260.154999  260.660004  260.100006   
1   4    JPM  2025-03-04 14:11:58  260.154999  260.660004  260.100006   
2   3    JPM  2025-03-04 14:10:57  260.154999  260.660004  260.100006   
3   2    JPM  2025-03-04 14:09:56  260.154999  260.660004  260.100006   
4   1    JPM  2025-03-04 14:08:53  260.154999  260.660004  260.100006   

        close  volume  
0  260.640015  248531  
1  260.640015  248531  
2  260.640015  248531  
3  260.640015  248531  
4  260.640015  248531  


In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL of the website
URL = "https://openlibrary.org/"

# Microsoft Edge User-Agent Header
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
}

def get_books(url):
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return []

    # Parse HTML content
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all book articles
    books = soup.find_all("article", class_="product_pod")
    book_list = []

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text.strip()  # Ensure clean price text
        stock = book.find("p", class_="instock availability").text.strip()

        book_list.append({"Title": title, "Price": price, "Availability": stock})

    return book_list

# Fetch book data
books_data = get_books(URL)

# Save data to CSV if books were found
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)
print("Data saved to books.csv")

Data saved to books.csv
